**Cargar librerias**

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import scipy as sc
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import sys

**Metricas**

In [2]:
from sklearn.metrics import roc_curve, auc
import sklearn as sk

    
def AMSScore(s,b): 
    return np.sqrt (2.*( (s + b + 10.)*np.log(1.+s/(b+10.))-s))

def eval_model(Y_true_train,Y_pred_train,w_train,Y_true_test,Y_pred_test,w_test):
    ratio =  float(len(X_train)) /float(len(X_test))
    TruePositive_train = w_train*(Y_true_train==1.0)*(1.0/ratio)
    TrueNegative_train = w_train*(Y_true_train==0.0)*(1.0/ratio)
    TruePositive_valid = w_test*(Y_true_test==1.0)*(1.0/(1-ratio))
    TrueNegative_valid = w_test*(Y_true_test==0.0)*(1.0/(1-ratio))
    s_train = sum ( TruePositive_train*(Y_pred_train==1.0) )
    b_train = sum ( TrueNegative_train*(Y_pred_train==1.0) )
    s_test = sum ( TruePositive_valid*(Y_pred_test==1.0) )
    b_test = sum ( TrueNegative_valid*(Y_pred_test==1.0) )
    score_train = AMSScore(s_train,b_train)
    score_test = AMSScore(s_test,b_test)
    print('--- Resultados --')
    print('- AUC train: {:.3f} '.format(sk.metrics.roc_auc_score(Y_train,Y_train_pred)))
    print('- AUC test : {:.3f} '.format(sk.metrics.roc_auc_score(Y_test,Y_test_pred)))
    print('- AMS train: {:.3f} sigma'.format(score_train))
    print('- AMS test : {:.3f} sigma'.format(score_test))
    return score_train, score_test

def plot_roc(clf,Y_test,Y_test_prob):
    fpr, tpr, thresholds = roc_curve(Y_test, Y_test_prob)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label=str(clf.__class__.__name__))
    plt.plot(np.linspace(0,1,100),np.linspace(0,1,100), lw=2, alpha=0.3, label='Suerte')
    plt.legend(loc='lower right')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.tight_layout()
    return

**Datos**

In [3]:
!wget https://raw.githubusercontent.com/beangoben/RIIAA_Escuela18/master/datos/training.csv

df=pd.read_csv('training.csv')
print(df.shape)
df.head(1)

'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: File b'training.csv' does not exist

**Preprocesamiento para X y Y**

Utilicé random_state para que en la division de los set se seleccionen los elementos de los grupos de forma aleatoria.

In [ ]:
Y = df['Label'].replace(to_replace=['s','b'],value=[1,0]).values
weights = df['Weight'].values
X = df.drop(['EventId','Label','Weight'],axis=1).values



In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test,w_train,w_test = train_test_split(X,Y,weights,random_state=42,train_size=0.3)
print(X_train.shape,Y_train.shape,w_train.shape)
print(X_test.shape,Y_test.shape,w_test.shape)

**ML con SKLearn**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

f = [('escalar', StandardScaler()),('polinomial', PolynomialFeatures(degree=2)),
    ('modelo', GradientBoostingClassifier(n_estimators=500, learning_rate=0.1,max_depth=3, random_state=0,verbose=1,subsample=0.7))]



pipeline = Pipeline(f)
pipeline.fit(X_train, Y_train)

**Predicciones**

In [4]:
Y_train_pred = pipeline.predict(X_train)
Y_test_pred = pipeline.predict(X_test)
Y_train_prob=pipeline.predict_proba(X_train)[:,1]
Y_test_prob =pipeline.predict_proba(X_test)[:,1]




NameError: name 'pipeline' is not defined

**Punto de corte**

In [5]:
sns.distplot(Y_train_prob)
plt.show()
pcut = np.percentile(Y_train_prob,85)
pcut

NameError: name 'Y_train_prob' is not defined

In [6]:
Y_train_pred = Y_train_prob > pcut 
Y_test_pred = Y_test_prob > pcut


eval_model(Y_train,Y_train_pred,w_train,Y_test,Y_test_pred,w_test)

NameError: name 'Y_train_prob' is not defined

In [7]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(Y_test, Y_test_prob)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=1, alpha=0.3, label=str(''))
plt.plot(np.linspace(0,1,100),np.linspace(0,1,100), lw=2, alpha=0.3, label='Suerte')
plt.legend(loc='lower right')
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('Falsos Positivos')
plt.ylabel('Falsos Negativos')
plt.tight_layout()
plt.show()

NameError: name 'Y_test' is not defined